# Inference on validation data

This notebook provides how to make prediction and visualize movement prediction to mp4 files.

Also you can stack prediction horizontally for comparing difference between prediction and real movements 

Default parameters: 

- FPS = 200
- NEW_FPS = 25
- WINDOW_SIZE = 512
- STRIDE = 512


In [1]:
import wandb, sys, os  
import numpy as np
import einops
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint


sys.path.insert(1, os.path.realpath(os.path.pardir))

from utils import data_utils, losses
from utils.hand_visualize import Hand, save_animation_mp4, visualize_and_save_anim, merge_two_videos, visualize_and_save_anim_gifs #, merge_two_videos_vertically
from utils.inference_utils import make_inference, calculcate_latency, get_angle_degree
from utils.quats_and_angles import get_quats, get_angles

from scipy import signal

from models import HVATNet_v2, HVATNet_v3, HVATNet_v3_FineTune

%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
        
def upload_weights_pl(model, path, pt):
    class Lit_Wrapper(pl.LightningModule):
        def __init__(self, model):
            super().__init__()
            self.model = model
        def forward(self, x):
            x = self.model(x)
            return x
    
    if pt == 'ckpt':
        ckpt = torch.load(path, map_location=torch.device('cpu'))
        model_pl = Lit_Wrapper(model)
        model_pl.load_state_dict(ckpt['state_dict'])
    else:
        try:
            ckpt = torch.load(path, map_location=torch.device('cpu'))
            model_pl = Lit_Wrapper(model)
            model_pl.load_state_dict(ckpt)
        except:
            ckpt = torch.load(path, map_location=torch.device('cpu'))
            model.load_state_dict(ckpt)
            return model
    
    return model_pl.model

In [3]:
class TrainConfig:
    WANDB_NOTES = 'HVATNet v3 FT train on all new data + no augs'

    datasets = ['../../data/general_set']
    # datasets = ['../../data/processed/dataset_v2_blocks']
    
    # hand_type = ['right']
    # human_type = ['health']
    hand_type = ['left']
    human_type = ['amputant']

    test_dataset_list = ['5']

    use_preproc_data = True # use preproc data (faster preparation  
    use_angles = True # use angels as target.
    
    original_fps = 250  # TODO describtion
    delay_ms = 0  # Shift vr vs EMG parameter. Do not work with preproc data. Fix it!!
    start_crop_ms = 0  # bad values in the beginning of recordign in ms to delete.
    window_size = 256
    down_sample_target = 8 # None

    max_epochs = 3000
    samples_per_epoch = 5000*256
    train_bs = 2048
    val_bs = 2048
    device = [2]
    optimizer_params = dict(lr=1e-4,
                            wd=0)
config = TrainConfig()

train_paths, val_paths = data_utils.get_train_val_pathes(config)

In [4]:
rootdir = Path('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Nastya/GeneralTraining/Train')
files = list(rootdir.glob('*'))

In [5]:
train_paths = files

In [6]:
rootdir = Path('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Nastya/GeneralTraining/Validation')
files = list(rootdir.glob('*'))

In [7]:
files

[WindowsPath('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Nastya/GeneralTraining/Validation/2_3'),
 WindowsPath('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Nastya/GeneralTraining/Validation/4_4'),
 WindowsPath('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Nastya/GeneralTraining/Validation/4_5'),
 WindowsPath('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Nastya/GeneralTraining/Validation/m3_5')]

In [8]:
val_paths = [files[0]]

In [9]:
val_paths

[WindowsPath('C:/Users/vlvdi/Desktop/EMG/EMG_TRAINING/Nastya/GeneralTraining/Validation/2_3')]

## Download data

In [10]:
# Init train and val dataset and dataloaders
val_datasets = []
for val_folder in val_paths:
    val_dataset = data_utils.create_dataset(data_folder=val_folder,
                                            original_fps=config.original_fps,
                                            delay_ms=config.delay_ms,
                                            start_crop_ms=config.start_crop_ms,
                                            window_size=config.window_size,

                                            down_sample_target = config.down_sample_target, 
                                            use_preproc_data=config.use_preproc_data, 
                                            use_angles=config.use_angles, 

                                            random_sampling = False,
                                            samples_per_epoch = None, 
                                            transform = None)
    
    val_datasets.append(val_dataset)

val_dataset = torch.utils.data.ConcatDataset(val_datasets)

Number of moves: 1 | Dataset: GeneralTraining


In [11]:
val_dataset = torch.utils.data.ConcatDataset(val_datasets)

In [12]:
MODEL_NAME = 'HVATNet_v3_small_unet_FT_angle'
DATA_TYPE = 'nast_best_model'
ANGLE_DEGREE = '13'

## Init Model

In [13]:
root = 'C:/Users/vlvdi/Desktop/EMG/alvi/weights/'

CKPT_PATH = root + 'pretrained_all_nast_6.016.pt'
#CKPT_PATH = artifact_dir / 'epoch=23_val_angle_degree=8.682.ckpt'

params =dict(n_electrodes=8, n_channels_out=20,
                        n_res_blocks=3, n_blocks_per_layer=3,
                        n_filters=128, kernel_size=3,
                        strides=(2, 2, 2),
                        dilation=2, use_angles = False)

model = HVATNet_v3_FineTune.HVATNetv3(**params)
model.use_angles = True

model = upload_weights_pl(model, CKPT_PATH, CKPT_PATH[-4:])

#model.load_state_dict(torch.load(CKPT_PATH, map_location=torch.device('cpu')))
model.eval()
print()

Number of parameters:  4210788



In [14]:
model.state_dict()['tune_module.spatial_weights']

tensor([[ 0.7824,  0.1518,  0.2810,  0.1888,  0.1750,  0.2631,  0.3175,  0.0570],
        [ 0.1442,  0.7647,  0.0708,  0.1398,  0.1881,  0.2690,  0.3261,  0.2829],
        [ 0.2185,  0.0531,  0.7363,  0.0091,  0.1638,  0.2452,  0.3222,  0.2985],
        [ 0.2124,  0.1643,  0.0782,  0.7879, -0.0666,  0.1827,  0.3160,  0.2695],
        [ 0.1967,  0.2016,  0.2038,  0.0171,  0.7882,  0.0988,  0.2837,  0.2546],
        [ 0.2245,  0.2594,  0.2554,  0.1814,  0.0082,  0.7817,  0.1932,  0.2614],
        [ 0.2868,  0.3239,  0.3231,  0.2942,  0.2764, -0.0134,  0.6971,  0.0507],
        [ 0.0888,  0.2627,  0.2954,  0.2264,  0.1954,  0.2002,  0.2157,  0.7339]])

## Apply inference for each moves.  

In [15]:
#torch.save(model.state_dict(), 'C:/Users/vlvdi/Desktop/EMG/alvi/model_nast_simple.pt')

In [16]:
val_dataset = torch.utils.data.ConcatDataset(val_datasets)

In [17]:
model = model.to('cpu')

In [20]:
FPS = 25
NEW_FPS = 25
DRAW_EVERY = 200 // NEW_FPS

WINDOW_SIZE = 256
STRIDE = 256

corr_list = []
angle_degree_list = []
preds_per_person = []
targets_per_person = []
abs_diff_per_person = []
angle_diff_per_person = []

for val_dataset_sample in val_dataset.datasets:
    
    all_move_targets = []
    all_move_preds = []
    all_myo = []
    
    for move_data in tqdm(val_dataset_sample.exps_data): 
        data_myo, data_vr = move_data['data_myo'], move_data['data_angles']
        print(data_myo.shape)
        preds, targets = make_inference(data_myo, data_vr, model,
                                        window_size=WINDOW_SIZE, 
                                        stride=STRIDE, 
                                        device='cpu', 
                                        return_angles=True)
        print(preds.shape)
        preds = get_angles(preds)
        print(preds.shape)
        if config.down_sample_target is not None:
            targets = targets[::config.down_sample_target]

        all_move_targets.append(targets)
        all_move_preds.append(preds)
        all_myo.append(data_myo)

    targets = np.concatenate(all_move_targets, axis=0)
    preds = np.concatenate(all_move_preds, axis=0)
    all_myo = np.concatenate(all_myo, axis=0)

    preds_per_person.append(all_move_preds)
    targets_per_person.append(all_move_targets)
    diff = np.abs(targets - preds)
    abs_diff_per_person.append(diff)
    mean_diff_angle_per_joint = np.rad2deg(np.mean(diff, axis = 1))
    angle_diff_per_person.append(mean_diff_angle_per_joint)
    
    # our metrics:
    dif = np.mean(diff)
    angle_degree = np.round(np.rad2deg(dif), 3)

    corr_coef = torch.mean(F.cosine_similarity(torch.from_numpy(targets), torch.from_numpy(preds), dim=0, eps=1e-8))
    corr_coef = np.round(corr_coef.item(), 3)
    
    corr_list.append(corr_coef)
    angle_degree_list.append(angle_degree)
    

# angle_degree = np.round(F.preds, targets), 3)

print(f"Size of targets {targets.shape} || Size of preds {preds.shape} ") 
print(f"Window size {WINDOW_SIZE} || Stride {STRIDE}")
print('Angle degree: ', np.mean(angle_degree_list), np.std(angle_degree_list))
print('Cosine similarity: ', np.mean(corr_list), np.std(corr_list))

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

(22240, 8)
(2752, 16, 4)


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.47s/it]

(2752, 20)
Size of targets (2752, 20) || Size of preds (2752, 20) 
Window size 256 || Stride 256
Angle degree:  8.889 0.0
Cosine similarity:  0.525 0.0


In [22]:
(22240, 8)
(2752, 16, 4)

(2752, 20)

8.081395348837209

In [56]:
pred_quats = get_quats(preds)
tar_quats = get_quats(targets)

In [55]:
NEW_FPS = 20
DRAW_EVERY = 3

In [58]:
visualize_and_save_anim(data=pred_quats[::DRAW_EVERY],
                        path=f'mar_preds_simple_tese.mp4', 
                        fps=NEW_FPS)

# visualize_and_save_anim(data=tar_quats[::DRAW_EVERY],
#                         path='targ_simple_thumb.mp4', 
#                         fps=NEW_FPS)

Video mar_preds_simple_tese.mp4 completed


In [60]:
plt.plot(pred_quats[:, [1, 2, 3, 4, 5, 6, 7, 10, 13, 14, 15], 0], color='lightcoral', alpha=0.75)
plt.plot(tar_quats[:, [1, 2, 3, 4, 5, 6, 7, 10, 13, 14, 15], 0], color='skyblue', alpha=0.75)
plt.xlabel('Time, samples', fontsize=12)
plt.ylabel('Scaled amplitude', fontsize=12)
plt.legend(['Prediction', 'Ground Truth'], fontsize=12)

ax = plt.gca()
leg = ax.get_legend()
leg.legendHandles[0].set_color('lightcoral')
leg.legendHandles[1].set_color('skyblue')

In [ ]:
# init_path = 'C:/Users/vlvdi/Desktop/EMG/MYO_DATA/Marusya/predictions/pretrained_arrays/'
# np.save(init_path + f'preds_simple_{finger}.npy', pred_quats)
# np.save(init_path + f'targ_simple_{finger}.npy', tar_quats)

In [20]:
# render and saving as gif files
visualize_and_save_anim_gifs(data=pred_quats[::DRAW_EVERY],
                        path=Path('C:/Users/vlvdi/Desktop/EMG/alvi/index_latest.gif'), 
                        fps=NEW_FPS)

NameError: name 'pred_quats' is not defined